In [1]:
import numpy as np
from keras import models
import time
import cv2
import tensorflow as tf

Using TensorFlow backend.
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packag

In [2]:
import numpy as np

def one_hot_decode(data):
    # 查找 one-hot 編碼中值為最大的索引
    max_index = np.argmax(data)
    
    # 獲取 one-hot 編碼中值為最大的數值
    max_value = np.max(data)
    
    # 返回最大值的索引和數值
    return int(max_index), float(max_value)



def translate_Y(yi , show = False):
    
    num = {
        0 : "paper", 1 : "rock", 2 : "scissor", } 
    
    yi_new = num.get( yi ) 
            
    if show:
        print(yi)
        print(yi_new)            
        
    return yi_new


In [3]:
name='MobileV2_bw.h5'
model = models.load_model(name)
print("load ",name)

# 選擇第攝影機
cap = cv2.VideoCapture(0)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
load  MobileV2_bw.h5


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_128 (Model)   (None, 3)                 3231939   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2048      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1539      
Total params: 3,235,526
Trainable params: 3,213,638
Non-trainable params: 21,888
_________________________________________________________________


In [4]:
while(True):
    # 從攝影機擷取一張影像

    ret, frame = cap.read()
    frame = frame/255
    # print(frame)
    
    # 模型預估
    frame_resize = cv2.resize(frame, (128, 128), interpolation=cv2.INTER_AREA)
    input_data=frame_resize[np.newaxis,:]
    # print(input_data.shape)
    start = time.time()
    prediction=model.predict(input_data) 
    end = time.time()
    accumulate_time = (end - start)
    FPS = 1/(accumulate_time+0.000001)
    FPS = round(FPS,2)
    result = one_hot_decode(prediction)
    # print(f"result = {result}")
    detected = translate_Y(result[0], show=True)
    #print(detected)
    
    
    # 顯示圖片
    cv2.putText(frame, str(FPS), (540, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, detected, (10, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
        
        # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()


ValueError: Error when checking input: expected input_2 to have shape (32, 32, 3) but got array with shape (128, 128, 3)

In [6]:
print(prediction)

[[0.18386367 0.16688433 0.649252  ]]


In [7]:
tf.__version__

'1.13.1'

In [8]:
tf.test.is_gpu_available()

False

In [9]:
cv2.__version__

'3.4.2'

In [10]:
np.__version__

'1.19.2'

# Run the code below

In [1]:
import numpy as np
from keras import models
import time
import cv2
import tensorflow as tf

def one_hot_decode(data):
    # 查找 one-hot 編碼中值為最大的索引
    max_index = np.argmax(data)
    
    # 獲取 one-hot 編碼中值為最大的數值
    max_value = np.max(data)
    
    # 返回最大值的索引和數值
    return int(max_index), float(max_value)



def translate_Y(yi , show = False):
    
    num = {
        0 : "paper", 1 : "rock", 2 : "scissor", } 
    
    yi_new = num.get( yi ) 
            
    if show:
        print(yi)
        print(yi_new)            
        
    return yi_new

name='MobileV2_bw.h5'
model = models.load_model(name)
print("load ",name)

# 選擇第攝影機
cap = cv2.VideoCapture(0)

while(True):
    # 從攝影機擷取一張影像

    fgbg = cv2.createBackgroundSubtractorMOG2()
    ret, frame = cap.read()
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, img = cv2.threshold(img, 155, 255, cv2.THRESH_BINARY)
    # frame = frame/255
    
    # 模型預估
    frame_resize = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    frame_resize = np.expand_dims(frame_resize, axis = 2)
    input_data=frame_resize[np.newaxis,:,:,:]
    # input_data = np.expand_dims(input_data, axis = 0)
    print(input_data.shape)
    start = time.time()
    prediction=model.predict(input_data/255) 
    end = time.time()
    accumulate_time = (end - start)
    FPS = 1/(accumulate_time+1e-12)
    FPS = round(FPS,2)
    result = one_hot_decode(prediction)
    # print(f"result = {result}")
    detected = translate_Y(result[0], show=True)
    #print(detected)
    
    
    # 顯示圖片
    cv2.putText(frame, str(FPS), (540, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, detected, (10, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
        
        # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()


Using TensorFlow backend.
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packag

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
load  MobileV2_bw.h5
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock


# Thres Binary

In [1]:
import numpy as np
from keras import models
import time
import cv2
import tensorflow as tf

def one_hot_decode(data):
    # 查找 one-hot 編碼中值為最大的索引
    max_index = np.argmax(data)
    
    # 獲取 one-hot 編碼中值為最大的數值
    max_value = np.max(data)
    
    # 返回最大值的索引和數值
    return int(max_index), float(max_value)



def translate_Y(yi , show = False):
    
    num = {
        0 : "paper", 1 : "rock", 2 : "scissor", } 
    
    yi_new = num.get( yi ) 
            
    if show:
        print(yi)
        print(yi_new)            
        
    return yi_new

name='MobileV2_bw.h5'
model = models.load_model(name)
print("load ",name)

# 選擇第攝影機
cap = cv2.VideoCapture(0)

while(True):
    # 從攝影機擷取一張影像

    fgbg = cv2.createBackgroundSubtractorMOG2()
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, frame = cv2.threshold(frame, 155, 255, cv2.THRESH_BINARY)
    # frame = frame/255
    
    # 模型預估
    frame_resize = cv2.resize(frame, (32, 32), interpolation=cv2.INTER_AREA)
    frame_resize = np.expand_dims(frame_resize, axis = 2)
    input_data=frame_resize[np.newaxis,:,:,:]
    # input_data = np.expand_dims(input_data, axis = 0)
    print(input_data.shape)
    start = time.time()
    prediction=model.predict(input_data/255) 
    end = time.time()
    accumulate_time = (end - start)
    FPS = 1/(accumulate_time+1e-12)
    FPS = round(FPS,2)
    result = one_hot_decode(prediction)
    # print(f"result = {result}")
    detected = translate_Y(result[0], show=True)
    #print(detected)
    
    
    # 顯示圖片
    cv2.putText(frame, str(FPS), (540, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, detected, (10, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
        
        # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()


Using TensorFlow backend.
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\USER\anaconda3\envs\MNIST\lib\site-packag

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
load  MobileV2_bw.h5
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
0
paper
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1, 32, 32, 1)
1
rock
(1,

In [10]:
while(True):
    # 從攝影機擷取一張影像

    fgbg = cv2.createBackgroundSubtractorMOG2()
    ret, frame = cap.read()
    frame = fgbg.apply(frame)
    print(frame.shape)
    frame_resize = cv2.resize(frame, (32, 32), interpolation=cv2.INTER_AREA)
    input_data=frame_resize[np.newaxis,:]
    # input_data = np.expand_dims(input_data, axis = 0)
    print(input_data.shape)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
while(True):
    # 從攝影機擷取一張影像

    fgbg = cv2.createBackgroundSubtractorMOG2()
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, th1 = cv2.threshold(frame, 150, 255, cv2.THRESH_BINARY)
    cv2.imshow('frame', th1)
    # input_data = np.expand_dims(input_data, axis = 0)
    # print(input_data.shape)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()